In [14]:
import os
import pandas as pd

dirProject = r"E:\GitHub\TDM-Bus-Speeds"
dirDataUTA = os.path.join(dirProject, r"data\UTA")

fnBusSpeeds = os.path.join(dirDataUTA, r"UTASep2019Tue-ThuBusSpeeds.xlsx")

In [19]:
from arcgis import GIS
gis = GIS()

In [16]:
dfBusSpeeds = pd.read_excel(fnBusSpeeds)
dfBusSpeeds

,ROUTE,DIR,period,STOP,StopAbbr,StopName,Timepoint,NodeAbbr,Lon,Lat,Avgmph,Avgmphdwell
0,2X,0,am peak,1,125213,Salt Lake Central Station (Bay B),True,SLCSTATN,-111.909334,40.763220,12.614483,11.812740
1,2X,0,am peak,2,126556,U President's Circle (SB Stop C),False,U.U.PRES,-111.852412,40.764553,7.914787,6.839896
2,2X,0,am peak,3,126261,Stadium Station (EB),False,STADIUM,-111.851398,40.760546,12.541677,9.097368
3,2X,0,am peak,4,126491,U Social Work Building (EB),False,NaN,-111.846654,40.760138,11.440429,10.101191
4,2X,0,am peak,5,101917,U Museum of Fine Arts (EB),False,NaN,-111.841221,40.759314,11.452424,9.059033
...,...,...,...,...,...,...,...,...,...,...,...,...
20610,F94,1,pm peak,32,101362,9800 S @ 2401 E,False,98-S24-E,-111.823917,40.573235,16.736905,0.000004
20611,F94,1,pm peak,32,101362,9800 S @ 2401 E,True,98-S24-E,-111.823917,40.573235,22.233332,0.000005
20612,F94,1,pm peak,33,198554,9800 S @ 2143 E,False,NaN,-111.829546,40.573250,24.648371,0.000004
20613,F94,1,pm peak,34,101361,9800 S @ 1991 E,False,NaN,-111.834346,40.573229,16.405984,0.000010


In [21]:
sdfBusSpeeds = pd.DataFrame.spatial.from_featureclass(strBusSpeedsOriginal)
sdfBusSpeeds

,FID,FromStopID,ToStopID,GRouteID,GRouteType,LineAbbr,LineName,FromGStopI,from_stop_,from_stop1,from_sto_1,from_sto_2,from_sto_3,ToGStopID,to_stop_co,to_stop_na,to_stop_de,to_stop_la,to_stop_lo,Shape_Leng,matchid,matchid_1,ROUTE,DIR,STOP,StopAbbr,StopName,Timepoint,NodeAbbr,NextStop,NextStop1,NextTimepo,NextNode,am_peak,midday,off_peak,pm_peak,am_peak_dw,midday_dwe,off_peak_d,pm_peak_dw,ObjectID,SHAPE
0,0,4173,5479,80327,3,205,500 EAST,21168,125332,Salt Lake Central Station,SALT LAKE CENTRAL STATION ST,40.763076,-111.909114,23807,198274,200 S @ 621 W,W 200 S,40.764901,-111.908917,203.279860,205-125332-198274,205-125332-198274,205,0.0,1.0,125332,Salt Lake Central Station (Bay A),1,SLCSTATN,198274,200 S / 600 W (EB),0,CTGPARK,7.249027,6.494686,6.747834,6.987258,5.258061,5.712941,6.143454,5.958174,339,"{""paths"": [[[423269.4968999997, 4512853.6031],..."
1,1,5479,127,80327,3,205,500 EAST,23807,198274,200 S @ 621 W,W 200 S,40.764901,-111.908917,5479,125021,200 S @ 255 W,W 200 S,40.764946,-111.898927,843.227379,205-198274-125021,205-198274-125021,205,0.0,2.0,198274,200 S / 600 W (EB),0,CTGPARK,125021,200 S / 300 W (EB),0,,10.095803,11.107568,12.302132,10.096893,9.823209,10.223701,11.866083,9.482716,430,"{""paths"": [[[423288.22300000023, 4513056.0186]..."
2,2,128,1399,80327,3,205,500 EAST,5481,125023,200 S @ 121 W,W 200 S,40.764883,-111.894684,13105,125404,200 S @ 9 W,W 200 S,40.764917,-111.891442,273.668787,205-125023-125404,205-125023-125404,205,0.0,5.0,125023,200 S / West Temple (EB),0,,125404,200 S / Main St (EB),0,MAIN2--S,11.329097,11.460183,11.865127,10.417450,7.295065,7.128774,7.950975,5.534769,338,"{""paths"": [[[424489.48500000034, 4513041.67380..."
3,3,1399,129,80327,3,205,500 EAST,13105,125404,200 S @ 9 W,W 200 S,40.764917,-111.891442,5482,126530,200 S @ 116 E,E 200 S,40.764899,-111.887530,330.200368,205-125404-126530,205-125404-126530,205,0.0,6.0,125404,200 S / Main St (EB),0,MAIN2--S,126530,200 S / State St (EB),1,2--SSTAT,8.163675,7.828876,7.987980,6.976040,7.422031,6.836966,7.064907,5.977564,340,"{""paths"": [[[424763.15199999977, 4513042.66290..."
4,4,129,1662,80327,3,205,500 EAST,5482,126530,200 S @ 116 E,E 200 S,40.764899,-111.887530,13636,125408,200 S @ 186 E,E 200 S,40.764898,-111.885659,157.922623,205-126530-125408,205-126530-125408,205,0.0,7.0,126530,200 S / State St (EB),1,2--SSTAT,125408,200 S / 200 E (EB),0,,9.314142,11.342662,10.032606,7.633491,7.979771,8.609253,9.261500,6.098318,380,"{""paths"": [[[425093.3091000002, 4513037.317600..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6149,6149,5277,5384,81587,3,871,TECH CORRIDOR RAIL CONNECTOR,23354,830036,CABELAS BLVD. @ 3723 N,CABELAS BLVD,40.433907,-111.882648,23620,198159,12300 S @ 192 E,E 12300 S,40.526641,-111.884616,10295.485119,871-830036-198159,871-830036-198159,871,1.0,5.0,830036,Cabela`s Blvd / Outlet Pkwy (EB),0,,198159,12300 S / 210 E (EB),0,123S3--E,29.760000,44.070000,0.000000,40.280000,29.480000,43.500000,0.000000,39.310000,4225,"{""paths"": [[[425136.9910000004, 4476291.7959],..."
6150,6150,5384,5339,81587,3,871,TECH CORRIDOR RAIL CONNECTOR,23620,198159,12300 S @ 192 E,E 12300 S,40.526641,-111.884616,23524,198128,700 E @ 12346 S,S 700 E,40.526090,-111.871305,1129.185152,871-198159-198128,871-198159-198128,871,1.0,6.0,198159,12300 S / 210 E (EB),0,123S3--E,198128,700 E / 12300 S (SB),0,,23.240000,21.250000,0.000000,18.780000,22.500000,19.720000,0.000000,18.750000,4221,"{""paths"": [[[425073.2496999996, 4486587.083699..."
6151,6151,5339,4687,81587,3,871,TECH CORRIDOR RAIL CONNECTOR,23524,198128,700 E @ 12346 S,S 700 E,40.526090,-111.871305,22204,101527,PIONEER RD @ 1020 E,E 12400 S (PIONEER RD),40.525068,-111.862811,728.393052,871-198128-101527,871-198128-101527,871,1.0,7.0,198128,700 E / 12300 S (SB),0,,101527,Pioneer Rd / Fort St (EB),0,,20.180000,18.420000,0.000000,19.440000,19.930000,18.410000,0.000000,19.30000

In [63]:
arcpy.management.CopyFeatures("LinksWithRiders_v3"    , "TDMLinks")
arcpy.management.CopyFeatures("September2019BusSpeeds", "UTALinks")

<Result 'E:\\GitHub\\TDM-Bus-Speeds\\arcgispro\\BusSpeeds\\BusSpeeds.gdb\\UTALinks'>

In [64]:
codeblock_az = """
def GetAzimuthPolyline(shape):
    if (shape.lastPoint.Y - shape.firstPoint.Y) != 0:
        degree = math.degrees(math.atan2(( shape.lastPoint.X  - shape.firstPoint.X ),( shape.lastPoint.Y - shape.firstPoint.Y )))
        if degree < 0:
            return 360 + degree
        else:
            return degree
    else:
        if shape.lastPoint.X > shape.firstPoint.X:
            return 90
        else:
            return 270
"""

codeblock_dir = """
def quadrant(NorthAzimuth):
    if ((NorthAzimuth>=45) & (NorthAzimuth<135)):
        quad = "EB"
    elif ((NorthAzimuth>=135) & (NorthAzimuth<225)):
        quad = "SB"
    elif ((NorthAzimuth>=225) & (NorthAzimuth<315)):
        quad = "WB"
    else:
        quad = "NB"
    return quad"""

arcpy.management.AddField("TDMLinks", "Azimuth", "DOUBLE")
arcpy.management.CalculateField("TDMLinks", "Azimuth", "GetAzimuthPolyline(!SHAPE!)", "PYTHON3", codeblock_az)

arcpy.management.AddField("TDMLinks", "Direction", "TEXT")
arcpy.management.CalculateField("TDMLinks", "Direction", "quadrant(!Azimuth!)", "PYTHON3", codeblock_dir)

arcpy.management.AddField("UTALinks", "Azimuth", "DOUBLE")
arcpy.management.CalculateField("UTALinks", "Azimuth", "GetAzimuthPolyline(!SHAPE!)", "PYTHON3", codeblock_az)

arcpy.management.AddField("UTALinks", "Direction", "TEXT")
arcpy.management.CalculateField("UTALinks", "Direction", "quadrant(!Azimuth!)", "PYTHON3", codeblock_dir)

<Result 'UTALinks'>